In [1]:
from main import fileconvert, clean
import pandas as pd
from Triple import Triple
from tqdm import tqdm
from spacy.matcher import Matcher
import spacy
import scispacy
import re
pd.set_option('display.max_colwidth', 200)
%matplotlib inline
nlp = spacy.load('en_core_web_sm')

User input -> <class 'str'>


In [ ]:
# Keep domain data separate.
fileconvert(path_to_folder, path_to_csv_output)

In [2]:
# wiki_csv = "/Users/awenc/NUIM/CS440/KG_NLPSystem/data/wiki_sentences_v2.csv"
data_csv = "/Users/awenc/NUIM/CS440/KG_NLPSystem/data/sentences.csv"
psychology_csv = "/Users/awenc/NUIM/CS440/KG_NLPSystem/data/sentences_psychology.csv"

# wiki_sentences = pd.read_csv(wiki_csv)
data_sentences = pd.read_csv(data_csv)
psychology_sentences = pd.read_csv(psychology_csv)

# print(len(wiki_sentences['sentence']))
print(len(data_sentences['sentences']))
print(len(psychology_sentences['sentences']))


563
886


In [3]:
def entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [4]:
from spacy.util import filter_spans  
def relation(sentence):
    """
    Get relation within input sentenceence based on pattern provided.
    params: str - input of single sentenceence.
    """
    doc = nlp(sentence)

    # Matcher class object
    matcher = Matcher(nlp.vocab, validate=True)


    """
    Match 0 or more times / match 0 or 1 time(one relation in sencence?)
    Dodałem PROPN ale jeszcze nie przetestowałem.
    """
    pattern0=[{'POS': 'VERB', 'OP': '?'},
            {'POS': 'ADV', 'OP': '*'},
            {'OP': '*'}, # additional wildcard - match any text in between
            {'POS': 'VERB', 'OP': '+'}]
    pattern1 = [{'DEP':'ROOT'},
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},
            {'POS':'PROPN','OP':'?'},
            {'POS':'ADJ','OP':"?"}]
    pattern = [{'POS': 'VERB', 'OP': '?'},
           {'POS': 'ADV', 'OP': '*'},
           {'POS': 'AUX', 'OP': '*'},
           {'POS': 'VERB', 'OP': '+'}]
    pattern2 = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 
    # pattern = [{'POS': 'VERB', 'OP': '?'}, {'POS': 'ADV', 'OP': ''}, {'OP': ''}, {'POS': 'VERB', 'OP': '+'}]

    matcher.add("Verb phrase", [pattern2])

    # call the matcher to find matches 
    matches = matcher(doc)
    spans = [doc[start:end] for _, start, end in matches]

    res = filter_spans(spans)
    return res

In [7]:
entity_pairs = []
relations = []
for i in tqdm(psychology_sentences['sentences']):
    entity_pairs.append(entities(i))
    relations.append(relation(i))

data_entity_pairs = []
data_relations = []
for i in tqdm(data_sentences['sentences']):
    data_entity_pairs.append(i)
    data_relations.append(i)


100%|██████████| 563/563 [00:00<00:00, 1397274.05it/s]


In [5]:
relations2 = [relation(i) for i in tqdm(psychology_sentences['sentences'])]
entities2 = [entities(i) for i in tqdm(psychology_sentences['sentences'])]

data_entities = [entities(i) for i in tqdm(data_sentences['sentences'])]
data_relations = [relation(i) for i in tqdm(data_sentences['sentences'])]

100%|██████████| 563/563 [00:03<00:00, 177.91it/s]


In [7]:
abstract_twenty_two = "A method for recovering structure and motion from a set of scaled orthographic silhouette views with unconstrained camer motion is presented. The outer epipolar tangencies between six or more views are used simultaneously to recover the relaticve pose of the cameras asn hance the cisual hull of the object. The camera represtation proposed permits a closed form solution in the optimazation of the camera parametrers. The resulting system is applied to the problem of reconstructing aircraft in flight."
sent01 = "Bobby was delighted he got a part in the school play, even though the part was a small one."
sent02 = "Because my porridge had gone cold, I heated it in the microwave."
sent03 = ""

In [12]:
# doc = nlp("John and his son David went to the park after he found his keys. They spend 2 hours at the park and got icecream.")
doc = nlp(sent02)
data_entity_pairs = []
data_relations = []


for sent in doc.sents:
    print(sent)
    data_entity_pairs.append(entities(str(sent)))
    data_relations.append(relation(str(sent)))
    print(relation(str(sent)))
    
    print(str(entities(str(sent))))
    print(str(relation(str(sent))))
# entities()
object_ = [i[0] for i in data_entity_pairs]
subject_ = [i[1] for i in data_entity_pairs]
relations_ = [i for i in data_relations]

Because my porridge had gone cold, I heated it in the microwave.
[heated]
['I', 'microwave']
[heated]


In [9]:
# print(type(data_entity_pairs))
object_ = [i[0] for i in data_entities]
subject_ = [i[1] for i in data_entities]
relations_ = [i for i in data_relations]

In [10]:
f"Objects: {len(object_)} Relations: {len(relations_)} Subjects: {len(subject_)}"

'Objects: 563 Relations: 1 Subjects: 563'

In [14]:
from main import clean

for i in range(len(relations_)):
    output = str(object_[i]) + str(relations_[i]) + str(subject_[i])
    # output = clean(output)
    output = re.sub("\[",' ', str(output))
    output = re.sub("\]",' ', str(output))
    output = re.sub(' +',' ',str(output))
    print(output)



I heated microwave


In [15]:
pd.Series([str(i) for i in data_relations]).value_counts()[:50] 

[heated]    1
dtype: int64